In [1]:
import os
import time
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import tensorflow as tf
import numpy as np
from urllib.request import urlopen

from lib.tools_batch import *
from lib.tools_math import *
from lib.tools_sparse import *
from lib.tools_audio import *
from lib.contrib.audio_featurizer import AudioFeaturizer
from lib.contrib.audio import AudioSegment
from model903 import *
model_name = "v903"

def timeStamp2Num(timeStamp, rate):
    """
    timeStamp str: 00:00:01,879
    rate int: the sampling rate
    return int
    """
    secs, millisec = timeStamp.split(",")
    hour, minute, sec = secs.split(":")
    millisec = float(millisec)*0.001
    sec = float(hour)*3600+float(minute)*60+float(sec)
    num = int(rate*(sec+millisec))
    return num

pyParser = pinyinParser("lib/pinyinDictNoTone.pickle")
model = model(409)
af = AudioFeaturizer()

In [2]:
sess = tf.Session()

In [3]:
saver = tf.train.Saver()
saver.restore(sess, "models/"+model_name+"/"+model_name+"_0.ckpt")

INFO:tensorflow:Restoring parameters from models/v903/v903_0.ckpt


In [5]:
rate, data = read_wav("data/test.wav")
data = mergeChannels(data)
data = zero_padding_1d(data, 160240)
a_seg = AudioSegment(data, rate)
xs = np.transpose(np.array([af.featurize(a_seg)]), [0,2,1])

pred = model.predict(sess, xs)[0]
pred_dense = sparseTuples2dense(pred)
detected_line = []
for stuff in pred_dense[0]:
    if stuff!=-1:
        detected_line.append(stuff)
pinyin = pyParser.decodeIndices(detected_line, useUnderline = False)
print(pinyin)
response = urlopen("https://www.google.com/inputtools/request?ime=pinyin&ie=utf-8&oe=utf-8&app=translate&num=10&text="+pinyin)
html = response.read()
result = (html.decode('utf8')).split(",")[2][2:-1]
print(result)


tamenxiangmusuodingdezhuyaokehuqunjiushiyouhaiwaizhiyexuqiudehuaren
他们项目锁定的主要客户群就是有海外置业需求的华人


In [ ]:
subtitles = []
new_subtitles = []
srtfile = "data/yixi_original.srt"
srtfile_recognized = "data/yixi.srt"
wavfile = "data/yixi.wav"

rate, data = read_wav(wavfile)

with open(srtfile, 'r') as f:
    for line in f:
        subtitles.append(line.strip())

j = 0
for i in range(len(subtitles)):
    print(str(i)+"/"+str(len(subtitles)))
    if "-->" in subtitles[i]:
        time1, time2 = [timeStamp2Num(x, rate) for x in subtitles[i].split(" --> ")]
        time1 = max([0, time1-8000])
        time2 = time2+8000
        print("Audio time = "+str((time2-time1)/16000.0)+" sec.")
        
        recog_starting_time = time.time()
        currentdata = data[time1:time2]
        if len(currentdata)<100:
            continue
        currentdata = mergeChannels(currentdata)
        if len(currentdata)>160240:
            j+=1
            new_subtitles.append(str(j))
            new_subtitles.append(subtitles[i])
            new_subtitles.append("语句太长，识别失败")
            new_subtitles.append("")
            continue
        currentdata = zero_padding_1d(currentdata, 160240)
        a_seg = AudioSegment(currentdata, rate)
        xs = np.transpose(np.array([af.featurize(a_seg)]), [0,2,1])
        
        pred = model.predict(sess, xs)[0]
        pred_dense = sparseTuples2dense(pred)
        detected_line = []
        for stuff in pred_dense[0]:
            if stuff!=-1:
                detected_line.append(stuff)
        if len(detected_line)<1:
            continue
        pinyin = pyParser.decodeIndices(detected_line, useUnderline = False)
        response = urlopen("https://www.google.com/inputtools/request?ime=pinyin&ie=utf-8&oe=utf-8&app=translate&num=10&text="+pinyin)
        html = response.read()
        result = (html.decode('utf8')).split(",")[2][2:-1]
        
        print("Recognition time = "+str(time.time()-recog_starting_time)+" sec.")
        print(result)
        print(subtitles[i+1])
        
        j+=1
        new_subtitles.append(str(j))
        new_subtitles.append(subtitles[i])
        new_subtitles.append(result)
        new_subtitles.append("")

new_subtitles = new_subtitles[:-1]
with open(srtfile_recognized, 'w+') as f:
    for line in new_subtitles:
        f.write(line+"\n")
        

0/2707
1/2707
Audio time = 3.0 sec.
2/2707
3/2707
4/2707
5/2707
6/2707
Audio time = 1.93 sec.
Recognition time = 8.552099704742432 sec.
大家好
大家好
7/2707
8/2707
9/2707
10/2707
Audio time = 3.5 sec.
Recognition time = 11.099024057388306 sec.
或持来自华中科学大学的章洛伊
我是来自华中科技大学的张珞颖
11/2707
12/2707
13/2707
14/2707
Audio time = 2.7 sec.
Recognition time = 11.181823015213013 sec.
我今天给伊达家呆了一个官
我今天给大家带来一个
15/2707
16/2707
17/2707
18/2707
Audio time = 3.85 sec.
Recognition time = 11.755331039428711 sec.
在一个关于我们为什么要睡觉的演讲
关于我们为什么要睡觉的演讲
19/2707
20/2707
21/2707
22/2707
Audio time = 3.6 sec.
Recognition time = 9.625288963317871 sec.
我们已身中带来有三分之一的时间
我们一生中大概有三分之一的时间
23/2707
24/2707
25/2707
26/2707
Audio time = 2.7 sec.
Recognition time = 9.863281965255737 sec.
是在睡眠中度过的
是在睡眠中度过的
27/2707
28/2707
29/2707
30/2707
31/2707
Audio time = 2.87 sec.
Recognition time = 8.321897983551025 sec.
你从出生到现在
那么你从出生到现在
32/2707
33/2707
34/2707
35/2707
Audio time = 2.83 sec.
Recognition time = 8.540001153945923 sec.
先大概睡了能有十年的叫了
大概睡了能有1

Recognition time = 10.043828964233398 sec.
睡眠又会被分为动眼睡眠和不动眼睡眠
睡眠又会被分为动眼睡眠和不动眼睡眠
269/2707
270/2707
271/2707
272/2707
Audio time = 3.13 sec.
Recognition time = 9.584776163101196 sec.
动睡眠不说我们人类可有的
动眼睡眠不光是我们人类特有的
273/2707
274/2707
275/2707
276/2707
Audio time = 3.05 sec.
Recognition time = 10.095137119293213 sec.
我们通常我所说的几个做梦
我们通常所说的做梦
277/2707
278/2707
279/2707
280/2707
281/2707
Audio time = 4.1 sec.
Recognition time = 9.666446208953857 sec.
大家可能觉得做工很高阶的中线向
大家可能觉得做梦是很高级的一种现象
282/2707
283/2707
284/2707
285/2707
Audio time = 3.43 sec.
Recognition time = 9.769724130630493 sec.
但是在骑士东野的水里面这个阶段
但是其实动眼睡眠这个阶段
286/2707
287/2707
288/2707
289/2707
Audio time = 4.15 sec.
Recognition time = 10.027660131454468 sec.
在一些爬行类动物在一些鱼类究竟
在一些爬行类动物 在一些鱼类
290/2707
291/2707
292/2707
293/2707
Audio time = 2.18 sec.
Recognition time = 11.577821254730225 sec.
就已经拍时出现了
就已经开始出现了
294/2707
295/2707
296/2707
297/2707
Audio time = 2.95 sec.
Recognition time = 11.365415811538696 sec.
所以可能不止我们人会做梦
所以可能不只我们人会做梦
298/2707
299/

Recognition time = 9.830487966537476 sec.
它分泌右边这个图里的这种叫做
它分泌右边这个图里的这种
524/2707
525/2707
526/2707
527/2707
Audio time = 2.64 sec.
Recognition time = 9.749634981155396 sec.
焦作市欲诉的起诉
叫做食欲素的激素
528/2707
529/2707
530/2707
531/2707
Audio time = 2.9 sec.
Recognition time = 9.705573797225952 sec.
这个时日苏轼一个堕胎
这个食欲素是一个多肽
532/2707
533/2707
534/2707
535/2707
Audio time = 2.98 sec.
Recognition time = 9.728498935699463 sec.
然后他又出轻型的作用
然后它有促清醒的作用
536/2707
537/2707
538/2707
539/2707
Audio time = 5.42 sec.
Recognition time = 9.726438999176025 sec.
所以在我们脑内分别是语素的脑区
所以在我们脑内分泌这个食欲素的脑区
540/2707
541/2707
542/2707
543/2707
Audio time = 2.28 sec.
Recognition time = 9.878099918365479 sec.
如果出现了问题
如果出现了问题
544/2707
545/2707
546/2707
547/2707
Audio time = 5.27 sec.
Recognition time = 9.774406909942627 sec.
让这个是匀速不能正常分离了就回的嗜睡症
让这个食欲素不能正常分泌了就会得嗜睡症
548/2707
549/2707
550/2707
551/2707
Audio time = 5.25 sec.
Recognition time = 9.71791696548462 sec.
如果反过来内邪从水面的脑区出现问题的话
如果反过来那些促睡眠的脑区出现问题的话
552/2707
553/2707
554/2707
555/27

Recognition time = 9.539638996124268 sec.
所以如果你一夜没睡熬夜了
所以如果你一夜没睡 熬夜了
783/2707
784/2707
785/2707
786/2707
Audio time = 3.87 sec.
Recognition time = 9.81445574760437 sec.
离第二天就会睡得比平时要酒喝声
你第二天就会睡得比平时要久和深
787/2707
788/2707
789/2707
790/2707
Audio time = 3.68 sec.
Recognition time = 9.829972982406616 sec.
然后如果说这个工作日睡的比较少
如果说你工作日睡得比较少
791/2707
792/2707
793/2707
794/2707
Audio time = 3.4 sec.
Recognition time = 9.848114728927612 sec.
那周末可能就会要不教会
那周末可能就会要补觉
795/2707
796/2707
797/2707
798/2707
Audio time = 3.52 sec.
Recognition time = 10.671857118606567 sec.
回水到特别的就特别的深
会睡得特别地久 特别地深
799/2707
800/2707
801/2707
802/2707
Audio time = 3.15 sec.
Recognition time = 11.05112338066101 sec.
可以所以从这里我们可以看出来
所以从这里我们可以看出来
803/2707
804/2707
805/2707
806/2707
Audio time = 3.93 sec.
Recognition time = 11.739706039428711 sec.
七十个睡眠呀离婚者所睡眠债一个
其实这个睡眠压力或者说睡眠债
807/2707
808/2707
809/2707
810/2707
Audio time = 3.92 sec.
Recognition time = 11.164644002914429 sec.
再以跟我们签的所有其他债一样都是要还的
跟我们欠的所有其他债一样 都是要还的
811/2707
812/270

Recognition time = 8.69693899154663 sec.
开始不是真的早睡早起了
看是不是真的早睡早起了
1042/2707
1043/2707
1044/2707
1045/2707
Audio time = 3.78 sec.
Recognition time = 10.330029964447021 sec.
所以究竟写了用脑电波的对睡眠的测量
所以就进行了用脑电波对睡眠的测量
1046/2707
1047/2707
1048/2707
1049/2707
Audio time = 5.17 sec.
Recognition time = 9.376667737960815 sec.
然后发现这些人他们的睡眠确实如白婷诉说
然后发现这些人他们的睡眠确实如Becky所说
1050/2707
1051/2707
1052/2707
1053/2707
Audio time = 5.05 sec.
Recognition time = 10.506406307220459 sec.
然后这个是比正常的人要造了带盖三个半到四个小时
这个是比正常的人要早大概3个半到4个小时
1054/2707
1055/2707
1056/2707
1057/2707
Audio time = 4.35 sec.
Recognition time = 9.216369152069092 sec.
因为这个现象是这个家族都有
因为这个现象是整个家族都有
1058/2707
1059/2707
1060/2707
1061/2707
Audio time = 3.5 sec.
Recognition time = 11.876338958740234 sec.
所以你就是说它是一个可以溢出的现象
所以就是说它是一个可以遗传的现象
1062/2707
1063/2707
1064/2707
1065/2707
Audio time = 4.98 sec.
Recognition time = 10.549348831176758 sec.
这位睡眠一生的就决定进一步的去调查这个事情
这位睡眠医生就决定进一步地去调查这个事情
1066/2707
1067/2707
1068/2707
1069/2707
Audio time = 2.92 sec.
Recognitio

Recognition time = 10.24026894569397 sec.
叫做季节性情感障碍
叫做季节性情感障碍
1292/2707
1293/2707
1294/2707
1295/2707
Audio time = 2.52 sec.
Recognition time = 9.906393051147461 sec.
俗称东徙倚欲
俗称冬季抑郁
1296/2707
1297/2707
1298/2707
1299/2707
Audio time = 3.5 sec.
Recognition time = 10.47254991531372 sec.
这个病的主要特征就是
这个病的主要特征就是
1300/2707
1301/2707
1302/2707
1303/2707
Audio time = 5.93 sec.
Recognition time = 10.54602599143982 sec.
就是患者在每年的秋天会开始出现一些抑郁的症状
患者在每年的秋天会开始出现一些抑郁的症状
1304/2707
1305/2707
1306/2707
1307/2707
Audio time = 4.45 sec.
Recognition time = 9.978378057479858 sec.
然后到冬天冻气的时候是最严重的
然后到冬季的时候是最严重的
1308/2707
1309/2707
1310/2707
1311/2707
Audio time = 3.72 sec.
Recognition time = 10.40015697479248 sec.
渠道每年十二月一月二月的时候是随眼中的
就到每年12月 1月 2月的时候是最严重的
1312/2707
1313/2707
1314/2707
1315/2707
Audio time = 3.75 sec.
Recognition time = 10.24106478691101 sec.
然后到次年的春夏优惠此法的好准
然后到次年的春夏又会自发地好转
1316/2707
1317/2707
1318/2707
1319/2707
Audio time = 4.06 sec.
Recognition time = 10.237606048583984 sec.
就是这一些性感障碍她在人群中的发病率


Recognition time = 9.738590955734253 sec.
试听了很多道理但是还是睡不了一个好觉
是听了很多道理 但是还是睡不了一个好觉
1536/2707
1537/2707
1538/2707
1539/2707
Audio time = 6.68 sec.
Recognition time = 11.137609958648682 sec.
所以就接下来的和大家聊一下一些广为流传的盖上
所以接下来和大家聊一下一些广为流传的
1540/2707
1541/2707
1542/2707
1543/2707
Audio time = 2.97 sec.
Recognition time = 10.444470167160034 sec.
改善或者促进睡眠的一些方法
改善或者促进睡眠的一些方法
1544/2707
1545/2707
1546/2707
1547/2707
Audio time = 2.4 sec.
Recognition time = 9.709181070327759 sec.
他们是不是真的有效
它们是不是真的有效
1548/2707
1549/2707
1550/2707
1551/2707
Audio time = 5.37 sec.
Recognition time = 10.36344313621521 sec.
大家可能都是听说过就是喝牛奶可以促进睡眠
大家可能都听说过喝牛奶可以促进睡眠
1552/2707
1553/2707
1554/2707
1555/2707
Audio time = 3.68 sec.
Recognition time = 9.31540298461914 sec.
喝着来是不是真的可以促进睡眠呢
喝牛奶是不是真的可以促进睡眠呢
1556/2707
1557/2707
1558/2707
1559/2707
Audio time = 3.87 sec.
Recognition time = 10.648298978805542 sec.
牛奶以及其它的一切乳制品里面
牛奶以及其他的一些乳制品里面
1560/2707
1561/2707
1562/2707
1563/2707
Audio time = 3.08 sec.
Recognition time = 9.4199099540710

Recognition time = 10.041284084320068 sec.
甚至是不能保护太子记得
甚至是不能保护它自己的
1787/2707
1788/2707
1789/2707
1790/2707
Audio time = 2.25 sec.
Recognition time = 9.727653980255127 sec.
但是基本是这辆的
但是即便是这样
1791/2707
1792/2707
1793/2707
1794/2707
Audio time = 2.48 sec.
Recognition time = 9.970431804656982 sec.
这两根我又睡了记忆点的叫了
动物也睡了几亿年的觉了
1795/2707
1796/2707
1797/2707
1798/2707
Audio time = 2.47 sec.
Recognition time = 10.636347770690918 sec.
这么说的话
这么说的话
1799/2707
1800/2707
1801/2707
1802/2707
Audio time = 3.15 sec.
Recognition time = 9.70245909690857 sec.
睡眠都不要付出那么大的代价
动物要付出那么大的代价
1803/2707
1804/2707
1805/2707
1806/2707
1807/2707
Audio time = 3.33 sec.
Recognition time = 10.953075170516968 sec.
所以睡眠应该是挺重要的
所以睡眠应该是挺重要的
1808/2707
1809/2707
1810/2707
1811/2707
Audio time = 2.55 sec.
Recognition time = 10.447063207626343 sec.
对于我们人类
对于我们人类
1812/2707
1813/2707
1814/2707
1815/2707
Audio time = 2.85 sec.
Recognition time = 10.32799506187439 sec.
如果一晚上的睡眠不足
如果一晚上的睡眠不足
1816/2707
1817/2707
1818/2707
1819/2707
Audio

In [ ]:
# 去掉空格，改数字，英文，和“他，她，它”
subtitles = []
new_subtitles = []
srtfile = "data/zongli_original.srt"
srtfile_recognized = "data/zongli.srt"
wavfile = "data/zongli.wav"

rate, data = read_wav(wavfile)

with open(srtfile, 'r') as f:
    for line in f:
        subtitles.append(line.strip())

j = 0
for i in range(len(subtitles))):
    print(str(i)+"/"+str(len(subtitles)))
    if "-->" in subtitles[i]:
        time1, time2 = [timeStamp2Num(x, rate) for x in subtitles[i].split(" --> ")]
        time1 = max([0, time1-8000])
        time2 = time2+8000
        
        currentdata = data[time1:time2]
        if len(currentdata)<100:
            continue
        currentdata = mergeChannels(currentdata)
        if len(currentdata)>160240:
            j+=1
            new_subtitles.append(str(j))
            new_subtitles.append(subtitles[i])
            new_subtitles.append(subtitles[i+1])
            new_subtitles.append("")
            continue
        currentdata = zero_padding_1d(currentdata, 160240)
        a_seg = AudioSegment(currentdata, rate)
        xs = np.transpose(np.array([af.featurize(a_seg)]), [0,2,1])
        
        pred = model.predict(sess, xs)[0]
        pred_dense = sparseTuples2dense(pred)
        detected_line = []
        for stuff in pred_dense[0]:
            if stuff!=-1:
                detected_line.append(stuff)
        if len(detected_line)<1:
            continue
        pinyin = pyParser.decodeIndices(detected_line, useUnderline = False)
        response = urlopen("https://www.google.com/inputtools/request?ime=pinyin&ie=utf-8&oe=utf-8&app=translate&num=10&text="+pinyin)
        html = response.read()
        result = (html.decode('utf8')).split(",")[2][2:-1]
        
        print(result)
        print(subtitles[i+1])
        
        j+=1
        new_subtitles.append(str(j))
        new_subtitles.append(subtitles[i])
        new_subtitles.append(result)
        new_subtitles.append("")

new_subtitles = new_subtitles[:-1]
with open(srtfile_recognized, 'w+') as f:
    for line in new_subtitles:
        f.write(line+"\n")